### 预处理

In [1]:
import sys

crack_results = {}
ns = []
es = []
cs = []

ns_es_cs = []
for i in range(21):
    with open(sys.path[0] + "/附件3-2（发布截获数据）/Frame" + str(i), 'r') as cipher_file:
        n_e_c = cipher_file.read()
        # 去除重复帧
        if ns_es_cs.count(n_e_c) == 0:
            ns_es_cs.append(n_e_c)
            ns.append(int(n_e_c[:256], 16))
            es.append(int(n_e_c[256:512], 16))
            cs.append(int(n_e_c[512:], 16))
            
num_of_frames = len(ns_es_cs)

### 因素碰撞

In [2]:
import gmpy2
from Crypto.Util.number import long_to_bytes

print("遍历所有N，若存在N之间有公因数，则存在因素碰撞攻击:")
same_factor_objs = []
for i in range(num_of_frames):
    for j in range(i + 1, num_of_frames):
        # n之间不可能存在倍数关系，但是要检查是否相等，相等则使用共模攻击
        if (ns[i] != ns[j]) and (gmpy2.gcd(ns[i], ns[j]) != 1):
            same_factor_objs.append((i, j))
print("[Obj]", same_factor_objs)

def crack_on_same_factor(same_factor_objs):
    for round in range(len(same_factor_objs)):
        p = gmpy2.gcd(ns[same_factor_objs[round][0]], ns[same_factor_objs[round][1]])
        for i in range(2):
            index = same_factor_objs[round][i]
            q = ns[index] // p
            varphi_n = gmpy2.mpz((p - 1) * (q - 1))
            d = gmpy2.invert(es[index], varphi_n)
            int_m = pow(cs[index], d, ns[index])
            str_m = long_to_bytes(int_m)[-8:].decode()
            crack_results[index] = str_m
            print("[Crack] frame {} -> m = {}".format(index, str_m))
            
crack_on_same_factor(same_factor_objs)

遍历所有N，若存在N之间有公因数，则存在因素碰撞攻击:
[Obj] [(1, 18)]
[Crack] frame 1 -> m = . Imagin
[Crack] frame 18 -> m = m A to B


### 共模攻击
假设使用相同模数时，是对相同的明文进行加密

In [3]:
from Crypto.Util.number import long_to_bytes

print("遍历所有模数N，存在重复N，且它们e不同则可能存在共模攻击:")
scanned = []
same_module_objs= []
for i in range(num_of_frames):
    if scanned.count(ns[i]) == 0:
        attack_candidate = [i]
        for j in range(i + 1, num_of_frames):
            if (ns[i] == ns[j]) and (es[i] != es[j]):
                attack_candidate.append(j)
    if len(attack_candidate) > 1:
        same_module_objs.append(attack_candidate)
print("[Obj]", same_module_objs)

# 欧几里得算法
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def crack_on_same_module(same_module_objs):
    for round in range(len(same_module_objs)):
        index0 = same_module_objs[round][0]
        index1 = same_module_objs[round][1]
        g, r, s = egcd(es[index0], es[index1])
        assert r * es[index0] + s * es[index1] == g
        int_m = pow(cs[index0], r, ns[index0]) * pow(cs[index1], s, ns[index0]) % ns[index0]
        str_m = long_to_bytes(int_m)[-8:].decode()
        crack_results[index0] = str_m
        crack_results[index1] = str_m
        print("[Crack] Frame {} -> m = {}".format(same_module_objs[round], str_m))
    return

crack_on_same_module(same_module_objs)

遍历所有模数N，存在重复N，且它们e不同则可能存在共模攻击:
[Obj] [[0, 4]]
[Crack] Frame [0, 4] -> m = My secre


### 低加密指数广播攻击

In [4]:
import gmpy2
from itertools import combinations

print("遍历所有指数e，如果存在e小于10，则可能成为低加密指数广播的攻击目标")
low_power_objs = {}
for i in range(num_of_frames):
    if es[i] <= 10:
        if low_power_objs.get(es[i]) == None:
            low_power_objs[es[i]] = [i]
        else:
            low_power_objs[es[i]].append(i)
print("[Objs]", low_power_objs)

def chinese_remainder_theorem(a_list, m_list):
    for (x, y) in combinations(m_list, 2):
        assert gmpy2.gcd(x, y) == 1
    m = 1
    result = 0
    for mi in m_list:
        m *= mi
    for i in range(len(m_list)):
        Mi = m // m_list[i]
        Mi_re = egcd(Mi, m_list[i])[1]
        result += Mi * Mi_re * a_list[i]
    return result % m

def crack_on_low_power(low_power_objs):
    for power in low_power_objs.keys():
        seqs = low_power_objs[power]
        c_list = [cs[i] for i in seqs]
        n_list = [ns[i] for i in seqs]
        m_pow = chinese_remainder_theorem(c_list, n_list)
        int_m = gmpy2.iroot(gmpy2.mpz(m_pow), power)
        str_m = long_to_bytes(int_m[0])[-8:].decode()
        for i in seqs:
            crack_results[i] = str_m
        print("[Crack] frame {} -> m = {}".format(seqs, str_m))
        return

crack_on_low_power(low_power_objs)

遍历所有指数e，如果存在e小于10，则可能成为低加密指数广播的攻击目标
[Objs] {5: [3, 8, 12, 16, 20], 3: [7, 11, 15]}
[Crack] frame [3, 8, 12, 16, 20] -> m = t is a f


### 费马分解法

In [5]:
def crack_on_fermat_resolve():
    for i in range(num_of_frames):
        x = gmpy2.isqrt(ns[i]) + 1
        y = gmpy2.isqrt(x * x - ns[i])
        count = 0
        while (x * x - ns[i] != y * y) and count < 100000:
            x += 1
            y = gmpy2.isqrt(x * x - ns[i])
            count += 1
        if x * x - ns[i] == y * y:
            p = (x + y) % ns[i]
            q = (x - y) % ns[i]
            varphi_n = (p - 1) * (q - 1)
            d = gmpy2.invert(es[i], varphi_n)
            int_m = pow(cs[i], d, ns[i])
            str_m = long_to_bytes(int_m)[-8:].decode()
            crack_results[i] = str_m
            print("[Crack] frame {} -> m = {}".format(i, str_m))

crack_on_fermat_resolve()

[Crack] frame 10 -> m = will get


### Pollard p-1 分解

In [6]:
def pollard(n):
    m = 2   # gcd(2, n) = 1
    max_k = 200000
    for k in range(1, max_k + 1):
        m = pow(m, k, n)  # 2^(k!)
        if gmpy2.gcd(m - 1, n) != 1:
            return gmpy2.gcd(m - 1, n)
    return

def crack_on_pollard_resolve():
    for i in range(num_of_frames):
        p = pollard(ns[i])
        if p != None:
            q = ns[i] // p
            varphi_n = (p - 1) * (q - 1)
            d = gmpy2.invert(es[i], varphi_n)
            int_m = pow(cs[i], d, ns[i])
            str_m = long_to_bytes(int_m)[-8:].decode()
            crack_results[i] = str_m
            print("[Crack] frame {} -> m = {}".format(i, str_m))

crack_on_pollard_resolve()

[Crack] frame 2 -> m =  That is
[Crack] frame 6 -> m =  "Logic 
[Crack] frame 19 -> m = instein.


In [7]:
from multiprocessing import Pool, Manager

def pollard(i, temp_dict):
    n = ns[i]
    m = 2   # gcd(2, n) = 1
    max_k = 200000
    for k in range(1, max_k + 1):
        m = pow(m, k, n)  # 2^(k!)
        if gmpy2.gcd(m - 1, n) != 1:
            p = gmpy2.gcd(m - 1, n)
            q = n // p
            varphi_n = (p - 1) * (q - 1)
            d = gmpy2.invert(es[i], varphi_n)
            int_m = pow(cs[i], d, ns[i])
            str_m = long_to_bytes(int_m)[-8:].decode()
            # crack_results[i] = str_m
            temp_dict[n] = str_m
            print("[Crack] frame {} -> m = {}".format(i, str_m))
            return
    return

def crack_on_pollard_resolve():
    temp_dict = Manager().dict()
    po = Pool(4)
    for i in range(num_of_frames):
        po.apply_async(func=pollard, args=(i, temp_dict,))
    po.close()
    po.join()
    for i in temp_dict.keys():
        crack_results[i] = temp_dict[i]

crack_on_pollard_resolve()

[Crack] frame 2 -> m =  That is
[Crack] frame 6 -> m =  "Logic 
[Crack] frame 19 -> m = instein.


In [8]:
# 打印消息
for i in range(num_of_frames):
    if crack_results.get(i) == None:

        print("Frame {:>2d} -> {}".format(i, ''))
    else:
        print("Frame {:>2d} -> {}".format(i, crack_results[i]))

Frame  0 -> My secre
Frame  1 -> . Imagin
Frame  2 ->  That is
Frame  3 -> t is a f
Frame  4 -> My secre
Frame  5 -> 
Frame  6 ->  "Logic 
Frame  7 -> 
Frame  8 -> t is a f
Frame  9 -> 
Frame 10 -> will get
Frame 11 -> 
Frame 12 -> t is a f
Frame 13 -> 
Frame 14 -> 
Frame 15 -> 
Frame 16 -> t is a f
Frame 17 -> 
Frame 18 -> m A to B
Frame 19 -> instein.
Frame 20 -> t is a f
